In [ ]:
from flask import Flask, request, render_template
import pandas as pd
import re
import csv
import sqlite3

In [ ]:
app = Flask(__name__, template_folder='templateFiles')

**API di Bawah untuk Cleansing Text (WORKS)**

In [ ]:
@app.route('/')
def my_form():
    return render_template('text_cleansing.html')

In [ ]:
@app.route('/', methods=['POST'])
def my_form_post():
    text = request.form['text']
    clean_text = text.replace("USER","").replace("RT","").replace("URL","").lower().strip()
    clean_text_2 = re.sub(r'[^a-zA-Z]',' ', clean_text)
    total_char = len(clean_text_2)
    total_word = len(clean_text_2.split())

    abusive_words = []
    with open('abusive.csv', 'r') as f:
        for x in f:
            if ";" in x:
                continue
            elif len(x.strip())>0:
                abusive_words.append(x.strip())

    jumlah_abusive = 0
    for kata in clean_text_2.split():
        for abusive_word in abusive_words:
            if kata == abusive_word:
                jumlah_abusive += 1
                break

    is_abusive_sentence = []
    if jumlah_abusive > 0:
        is_abusive_sentence.append("Kalimat Abusive")
    else:
        is_abusive_sentence.append("Bukan Kalimat Abusive")

    response = {
    'data' : clean_text_2.split(),
    'jumlah karakter' : total_char,
    'jumlah kata' : total_word,
    'jumlah abusive words' : jumlah_abusive,
    'kesimpulan' : is_abusive_sentence
    }
        
    return response

**API di Bawah Untuk Cleansing File CSV (WORKS) --> pake data_latihan.csv untuk try**

In [ ]:
@app.route('/csv-upload', methods=['GET', 'POST'])
def index():
    return render_template('index.html')

In [ ]:
@app.route('/data', methods=['GET', 'POST'])
def data():
    if request.method == 'POST':
        file = request.form.get('csvfile')
        #data = csvfile
        data = []
        #with open(f) as file:
        data = pd.read_csv(file)
            #for row in csvfile:
                #data.append(row)
        #data = pd.DataFrame(data)
        
        data['total_char'] = data['Tweet'].apply(len)
        data['total_word'] = data['Tweet'].apply(lambda x : len(x.split()))
        data['1'] = data['Tweet'].apply (lambda x : x.replace("USER", "").replace("RT", "").replace("URL", "").lower())
        data['2'] = data['1'].astype(str).str.replace(r'[^a-zA-Z]', ' ', regex=True)
        data['cleanse_tweet'] = data['2'].apply(lambda x:x.strip())
        data['new_cleanse_tweet'] = data['cleanse_tweet'].apply (lambda x :x.split())
        data['total_char_cleanse_tweet'] = data['cleanse_tweet'].apply(len)
        data['total_word_cleanse_tweet'] = data['cleanse_tweet'].apply(lambda x : len(x.split()))
        
        abusive_words = []
        with open('abusive.csv', 'r') as f:
            for x in f:
                if ";" in x:
                    continue
                elif len(x.strip())>0:
                    abusive_words.append(x.strip())
        
        unique_words = []
        for sentence in data['new_cleanse_tweet']:
            for word in sentence:
                unique_words.append(word)
                
        unique_words = list(set(unique_words))
                
        word_to_index = {}
        for index,word in enumerate(unique_words):
            word_to_index[word] = index
            
        index_to_word = {}
        for word,index in word_to_index.items():
            index_to_word[index] = word
        
        for word in abusive_words:
            try:
                word_to_index[word]
            except KeyError:
                word_to_index[word] = len(word_to_index)
                
        index_to_word = {}
        for word,index in word_to_index.items():
            index_to_word[index] = word
            
        new_df_words = []
        for sentence in data['new_cleanse_tweet']:
            new_sentence = []
            for word in sentence:
                new_sentence.append(word_to_index.get(word, -1))
            new_df_words.append(new_sentence)
            
        new_abusive_words = [word_to_index.get(word, -1) for word in abusive_words]
        
        jumlah_abusive_words_column = []
        for row in (new_df_words):
            jumlah_abusive_words = 0 
            for abusive_word in new_abusive_words:
                if abusive_word in row:
                    jumlah_abusive_words = jumlah_abusive_words + 1
            jumlah_abusive_words_column.append(jumlah_abusive_words)
            
        data['jumlah_abusive_words'] = jumlah_abusive_words_column
        
        is_abusive_sentence = []
        for sentence in data['jumlah_abusive_words']:
            if sentence > 0:
                is_abusive_sentence.append("Kalimat Abusive")
            else:
                is_abusive_sentence.append("Bukan Kalimat Abusive")
                
        data['is_abusive_sentence'] = is_abusive_sentence
        
        data = data[['Tweet', 'total_char', 'total_word', 'cleanse_tweet', 'new_cleanse_tweet', 'total_char_cleanse_tweet', 'total_word_cleanse_tweet', 'jumlah_abusive_words', 'is_abusive_sentence']]
        
        return render_template('data.html', data=data.to_html(header=True, index=False))

**API Untuk SQLite (masih BELUM WORKS (｡•́︿•̀｡))**

In [ ]:
conn = sqlite3.connect('cleansing_text.db')

In [ ]:
conn.execute('''
CREATE TABLE cleansing_text (text varchar(255) not null,
                            text_cleansing varchar(255),
                            total_char int,
                            total_word int,
                            jumlah_abusive_word int,
                            conclusion varchar(255)
);
''')
conn.commit()

In [ ]:
@app.route('/', methods=['GET', 'POST'])
def cleansing_text():
    msg = "msg"
    if request.method == "GET":
        try:
            teks = text
            data_cleansing = clean_text_2.split()
            jumlah_char = total_char
            jumlah_word = total_word
            jumlah_abusive_word = jumlah_abusive
            kesimpulan = is_abusive_sentence
            with sqlite3.connect('cleansing_text.db') as con:
                cur = con.cursor()
                cur.execute('''INSERT into cleansing_text (text, text_cleansing, total_char, total_word, jumlah_abusive_word, conclusion) values (?, ?, ?, ?, ?, ?)''', (teks, data_cleansing, jumlah_char, jumlah_word, jumlah_abusive_word, kesimpulan))
                con.commit()
        except:
            con.rollback()
        finally:
            con.close()

In [ ]:
if __name__ == "__main__":
    app.run()